[View in Colaboratory](https://colab.research.google.com/github/pmanlukas/colabSVM/blob/master/linear_classifier.ipynb)

# Linear Classifier
In this notebook a linear classifier based on a SVM is developed that is used as benchmark for the neural network based classifier

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

Setting up lsb-release (9.20160110ubuntu5) ...
Setting up libgirepository-1.0-1:amd64 (1.54.1-1) ...
Setting up libkmod2:amd64 (24-1ubuntu2) ...
Setting up gir1.2-glib-2.0:amd64 (1.54.1-1) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Setting up libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Setting up unattended-upgrades (0.98ubuntu1.1) ...

Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

In [5]:
!ls /content/drive/openapi-data/

glove.6B.zip  path_labels.csv  PoC Classifier Path.odt	weights.best.hdf5
mnist.py      pickle	       structure_labels.csv	wordvector


In [0]:
import os
os.chdir("drive/openapi-data")

In [7]:
!ls

glove.6B.zip  path_labels.csv  PoC Classifier Path.odt	weights.best.hdf5
mnist.py      pickle	       structure_labels.csv	wordvector


In [0]:
import pickle

In [0]:
training_data = dict()

with open('pickle/specjson210518.pkl', 'rb') as handle:
    training_data = pickle.load(handle)

In [0]:
keys = list(training_data.keys())

In [11]:
training_data[keys[0]]

{'basePath': '/forex-quotes',
 'host': '1forge.com',
 'info': {'contact': {'email': 'contact@1forge.com',
   'name': '1Forge',
   'url': 'http://1forge.com'},
  'description': 'Stock and Forex Data and Realtime Quotes',
  'title': '1Forge Finance APIs',
  'version': '0.0.1',
  'x-apisguru-categories': ['financial'],
  'x-logo': {'backgroundColor': '#24292e',
   'url': 'https://api.apis.guru/v2/cache/logo/http_1forge.com_logo.png'},
  'x-origin': [{'format': 'swagger',
    'url': 'http://1forge.com/openapi.json',
    'version': '2.0'}],
  'x-preferred': True,
  'x-providerName': '1forge.com'},
 'paths': {'/quotes': {'get': {'description': 'Get quotes',
    'externalDocs': {'description': 'Find out more',
     'url': 'http://1forge.com/forex-data-api'},
    'responses': {'200': {'description': 'A list of quotes'}},
    'summary': 'Get quotes for all symbols',
    'tags': ['forex', 'finance', 'quotes']}},
  '/symbols': {'get': {'description': 'Symbol List',
    'externalDocs': {'descripti

In [0]:
from collections import Counter
from datetime import datetime
 
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [0]:
labelsP = pd.read_csv("path_labels0518.csv").values
labelsS  = pd.read_csv("structure_labels0518.csv").values

In [14]:
texts = []   # list of text samples
for spec in training_data:
  strJson = json.dumps(training_data[spec])
  texts.append(strJson)

print(str(len(texts)))

1034


In [15]:
labelsPath = list()
labelsStruct = list()


for labs in labelsS:
  labelsStruct.append(labs[1])

for labp in labelsP:
  labelsPath.append(labp[1])
  
print(str(len(labelsPath)))
print(str(len(labelsStruct)))






1034
1034


In [0]:
from sklearn.model_selection import train_test_split

#split the data and labels into a test and train set
X_trainS, X_testS, y_trainS, y_testS = train_test_split(texts, labelsStruct, test_size=0.3, random_state=0)

X_trainS.shape, y_trainS.shape

X_testS.shape, y_testS.shape

In [0]:
#split the data and labels into a test and train set
X_trainP, X_testP, y_trainP, y_testP = train_test_split(texts, labelsPath, test_size=0.3, random_state=0)

X_trainP.shape, y_trainP.shape

X_testP.shape, y_testP.shape

## model architecture

the first model is for structure labels

In [16]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

 
#TODO: change code to use sklearn Pipeline
  
t1 = datetime.now()

#vectorizer based on TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=3)
tf_transformer = vectorizer.fit(texts)

XtrainS = tf_transformer.transform(X_trainS)
XtestS = tf_transformer.transform(X_testS)

#Classifiers, Naive Bayes and linear SVM based
clfSVC = LinearSVC()
clfNB = MultinomialNB()

print(datetime.now() - t1)
print(XtrainS.shape)

# TODO: adjust classifier to generate better accuracy
model_svm = clfSVC.fit(XtrainS, y_trainS)
model_nb = clfNB.fit(XtrainS, y_trainS)

#cross validation
score_svm = cross_val_score(model_svm, XtrainS, y_trainS, cv=10, n_jobs=-1)
score_nb = cross_val_score(model_nb, XtrainS, y_trainS, cv=10, n_jobs=-1)
 
print(datetime.now() - t1)
print("CV of SVM:")
print(score_svm)
print(sum(score_svm) / len(score_svm))
print("CV of nb:")
print(score_nb)
print(sum(score_nb) / len(score_nb))

0:00:23.099658
(1034, 122730)
0:00:43.225324
[0.75700935 0.80188679 0.82857143 0.84761905 0.75238095 0.7745098
 0.82352941 0.67647059 0.45544554 0.36363636]
0.708105927893104


In [0]:
# TODO: add test based on trained classifier
predictions_svm = cross_val_predict(model_svm, XtestS, y_testS, cv=10)
plt.scatter(y_testS, predictions_nb)

predictions_nb = cross_val_predict(model_nb, XtestS, y_testS, cv=10)
plt.scatter(y_testS, predictions_nb)

The following model is created for the paths label

In [17]:
#TODO: adjust model to be like for Strucutre 
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
 
t1 = datetime.now()
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=3)
classifierp = LinearSVC()
Xp = vectorizer.fit_transform(texts)
 
print(datetime.now() - t1)
print(Xp.shape)
 
score = cross_val_score(classifierp, Xp, labelsPath, cv=10, n_jobs=-1)
 
print(datetime.now() - t1)
print(score)
print(sum(score) / len(score))

0:00:22.043082
(1034, 122730)
0:00:33.808667
[0.56190476 0.63461538 0.69230769 0.61538462 0.53846154 0.39423077
 0.60194175 0.49019608 0.6372549  0.54901961]
0.5715317097712871


In [0]:
#TODO: add prediction for Path

## package the model

In [0]:
from sklearn.externals import joblib

In [0]:
# TODO: search for more efficient way to deserialize the vectorizer

joblib.dump(vectorizer, "pickle/tfidf_vectorizer.pickle")

In [21]:
joblib.dump(classifier, "pickle/svm_classifier.pickle")


['pickle/svm_classifier.pickle']

In [22]:
joblib.dump(classifierp, "pickle/svm_classifierP.pickle")

['pickle/svm_classifierP.pickle']